In [7]:
!sudo apt-get install xvfb

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common
The following NEW packages will be installed:
  libfontenc1 libxfont2 libxkbfile1 x11-xkb-utils xfonts-base xfonts-encodings
  xfonts-utils xserver-common xvfb
0 upgraded, 9 newly installed, 0 to remove and 49 not upgraded.
Need to get 7,813 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libfontenc1 amd64 1:1.1.4-1build3 [14.7 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxfont2 amd64 1:2.0.5-1build1 [94.5 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libxkbfile1 amd64 1:1.1.0-1build3 [71.8 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/main amd64 x11-xkb-utils amd64 7.7+5build4 [172 kB]
Get:5 http://archiv

In [9]:
!xvfb-run python your_script.py

python3: can't open file '/content/your_script.py': [Errno 2] No such file or directory


In [2]:
import requests
from tkinter import *

In [11]:
tl = Tk()
tl.title('Parametros VOD')
txt_stp = Label(tl, text = "coloque os parametros do VOD")
txt_stp.grid(column = 0 , row = 1)



tl.mainloop()

TclError: no display name and no $DISPLAY environment variable

In [ ]:
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.exceptions import ConvergenceWarning
import warnings

warnings.filterwarnings('ignore', category=ConvergenceWarning)

def run_algorithm_loop(ent_, contr, dt, time_limit=300, target_prob=0.99):
    """
    Runs a loop to process data and apply logistic regression until a certain condition is met.

    Parameters:
    - ent_: DataFrame, the first input data frame
    - contr: DataFrame, the second input data frame (to be updated)
    - dt: DataFrame, data for training the model
    - time_limit: int, the maximum allowed time for the loop to run (in seconds)
    - target_prob: float, the probability threshold to break the loop (default is 0.99)

    Returns:
    - updated contr DataFrame
    - results_df_: DataFrame with the impact results
    """
    # Initialize the loop variables
    condition_met = False
    start_time = time.time()
    iteration = 0

    while not condition_met:
        try:
            # Track elapsed time
            elapsed_time = time.time() - start_time
            if elapsed_time > time_limit:
                print("Time limit exceeded. Stopping the loop.")
                break

            iteration += 1
            print(f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds")

            # Reset index for contr DataFrame
            contr.reset_index(drop=True, inplace=True)

            # Concatenate and process DataFrames
            tt2 = pd.concat([ent_, contr], axis=1)
            tt2.fillna(0, inplace=True)
            tt2 = pd.DataFrame(tt2.sum()).T

            # Here, add the columns you want for your test
            tt2_test = tt2[[]]  # Define relevant columns

            # Extract feature and target data for the logistic regression
            x_ = dt[[]]  # Define relevant columns
            y_ = dt['opt']

            # Split data into training and test sets
            x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

            # Train logistic regression model
            model = LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced')
            model.fit(x_train_3, y_train_3)

            # Make predictions
            y_pred2 = model.predict(tt2)
            y_pred_prob2 = model.predict_proba(tt2)[:, 1]  # Get probability of the positive class
            y_pred_score2 = model.score(x_test_3, y_test_3)

            # Causal inference using Logit model
            model = sm.Logit(y_train_3, x_train_3)
            co = model.fit(method='bfgs', cov_type='HC3').params
            coe = pd.DataFrame(co, columns=['coeff'])
            exp_coeff = np.exp(coe) / (np.exp(coe) + 1)
            exp_coeff.columns = ['exp_coeff']

            results = model.fit(method='bfgs', cov_type='HC3')
            results_summary = results.summary()
            results_as_html = results_summary.tables[1].as_html()
            df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
            df_results['exp_coeff'] = exp_coeff
            impact_var2 = df_results.loc[df_results['P>|z|'] < 0.05]

            # Process significant variables and corrections
            results_list = []
            for i in impact_var2.index:
                value = impact_var2.loc[i, 'exp_coeff']
                if i in contr.columns:
                    correction = tt2_test[i].mean()
                    result = (value - 0.5) * correction if value != 0.5 else 0
                    results_list.append({'Column': i, 'Alteração': result})

            # Create results DataFrame
            results_df_ = pd.DataFrame(results_list)
            dd = results_df_.T
            new_header = dd.iloc[0]
            dd = dd[1:]
            dd.columns = new_header
            dd.reset_index(drop=True, inplace=True)

            # Combine and update contr DataFrame
            common_columns = list(set(contr.columns) & set(dd.columns))
            contr_common = contr[common_columns]
            dd_common = dd[common_columns]

            combined_df = pd.concat([contr_common, dd_common], ignore_index=True)
            final = combined_df.sum().reset_index()
            tt = final.T
            new_header = tt.iloc[0]
            tt = tt[1:]
            tt.columns = new_header
            tt.reset_index(drop=True, inplace=True)

            contr.update(tt[contr.columns.intersection(tt.columns)])
            contr.reset_index(drop=True, inplace=True)

            # Clean up memory
            gc.collect()

            # Check if the condition is met (all predicted probabilities >= target_prob and predicted class is 1)
            if np.all(y_pred_prob2 >= target_prob) and np.all(y_pred2 == 1):
                condition_met = True

        except Exception as e:
            print(f"An error occurred: {e}")
            # Log or handle the error, but continue the loop
            continue

    # After the loop ends, return the results
    return contr, results_df_

# Example call to the function:
# updated_contr, results_df = run_algorithm_loop(ent_, contr, dt, time_limit=300, target_prob=0.99)


In [ ]:
import tkinter as tk

# Function to be triggered when the button is pressed
def run_algorithm():
    # Get the parameters from the input fields
    param1 = entry_param1.get()
    param2 = entry_param2.get()

    # Validate inputs
    if not param1 or not param2:
        result_label.config(text="Please enter all parameters.")
        return

    try:
        # Convert inputs to floats (or another type as needed)
        param1 = float(param1)
        param2 = float(param2)

        # Dummy algorithm (replace with your actual algorithm logic)
        result = param1 + param2  # Example: summing the two parameters

        # Display the result in the label
        result_label.config(text=f"The result of the algorithm is: {result}")
    except ValueError:
        result_label.config(text="Please enter valid numbers.")

# Creating the main Tkinter window
tl = tk.Tk()
tl.title('Parametros VOD')

# Creating labels and input fields for parameters
label_param1 = tk.Label(tl, text="Parameter 1:")
label_param1.grid(column=0, row=0, padx=10, pady=10)

entry_param1 = tk.Entry(tl)
entry_param1.grid(column=1, row=0, padx=10, pady=10)

label_param2 = tk.Label(tl, text="Parameter 2:")
label_param2.grid(column=0, row=1, padx=10, pady=10)

entry_param2 = tk.Entry(tl)
entry_param2.grid(column=1, row=1, padx=10, pady=10)

# Button to activate the algorithm function
btn_run = tk.Button(tl, text="Run Algorithm", command=run_algorithm)
btn_run.grid(column=0, row=2, columnspan=2, padx=10, pady=10)

# Label to display the result
result_label = tk.Label(tl, text="")
result_label.grid(column=0, row=3, columnspan=2, padx=10, pady=10)

# Start the Tkinter main loop
tl.mainloop()


In [ ]:
def run_algorithm(stp_values):
    # Initialize the parameters and data
    columns_ = ['Column1', 'Column2', 'Column3', 'Column4', 'Column5', 'Column6']  # Adjust column names
    stp = [stp_values]
    contr = pd.DataFrame(stp, columns=columns_)
    contr.reset_index(drop=True, inplace=True)

    # Dummy data for dt (replace with actual dataset)
    dt = pd.DataFrame(np.random.rand(100, 6), columns=columns_)
    dt['opt'] = np.random.randint(0, 2, size=100)

    ent = dt.sample(n=1)
    ent_ = ent.copy()

    tt2 = pd.concat([ent_, contr], axis=1)
    tt2.fillna(0, inplace=True)
    tt2 = pd.DataFrame(tt2.sum()).T

    # Feature and target columns for training (adjust accordingly)
    x_ = dt[columns_]  # Features
    y_ = dt['opt']     # Target

    # Train/test split
    x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)

    # Logistic regression
    model = LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced')
    model.fit(x_train_3, y_train_3)
    y_pred2 = model.predict(tt2)
    y_pred_prob2 = model.predict_proba(tt2)

    # Causal inference using Logit model
    logit_model = sm.Logit(y_train_3, x_train_3)
    results = logit_model.fit(method='bfgs', cov_type='HC3')
    coe = pd.DataFrame(results.params, columns=['coeff'])
    exp_coeff = np.exp(coe) / (np.exp(coe) + 1)
    exp_coeff.columns = ['exp_coeff']

    results_summary = results.summary()
    results_as_html = results_summary.tables[1].as_html()
    df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
    df_results['exp_coeff'] = exp_coeff
    impact_var2 = df_results.loc[df_results['P>|z|'] < 0.05]

    return y_pred2, y_pred_prob2, impact_var2

def on_run_button():
    # Get parameters from the input fields
    param1 = float(entry_param1.get())
    param2 = float(entry_param2.get())
    param3 = float(entry_param3.get())
    param4 = float(entry_param4.get())
    param5 = float(entry_param5.get())
    param6 = float(entry_param6.get())

    stp_values = [param1, param2, param3, param4, param5, param6]

    # Run the algorithm and get the results
    y_pred2, y_pred_prob2, impact_var2 = run_algorithm(stp_values)

    # Display the results in the label
    result_text = f"Predicted Class: {y_pred2[0]}, Probability: {y_pred_prob2[0][1]:.2f}\n"
    result_text += f"Impact Variables:\n{impact_var2.to_string()}"
    result_label.config(text=result_text)

# Create the Tkinter window
tl = tk.Tk()
tl.title('Parametros VOD')

# Create input fields for the 6 parameters
label_param1 = tk.Label(tl, text="Parameter 1:")
label_param1.grid(column=0, row=0, padx=10, pady=5)
entry_param1 = tk.Entry(tl)
entry_param1.grid(column=1, row=0, padx=10, pady=5)

label_param2 = tk.Label(tl, text="Parameter 2:")
label_param2.grid(column=0, row=1, padx=10, pady=5)
entry_param2 = tk.Entry(tl)
entry_param2.grid(column=1, row=1, padx=10, pady=5)

label_param3 = tk.Label(tl, text="Parameter 3:")
label_param3.grid(column=0, row=2, padx=10, pady=5)
entry_param3 = tk.Entry(tl)
entry_param3.grid(column=1, row=2, padx=10, pady=5)

label_param4 = tk.Label(tl, text="Parameter 4:")
label_param4.grid(column=0, row=3, padx=10, pady=5)
entry_param4 = tk.Entry(tl)
entry_param4.grid(column=1, row=3, padx=10, pady=5)

label_param5 = tk.Label(tl, text="Parameter 5:")
label_param5.grid(column=0, row=4, padx=10, pady=5)
entry_param5 = tk.Entry(tl)
entry_param5.grid(column=1, row=4, padx=10, pady=5)

label_param6 = tk.Label(tl, text="Parameter 6:")
label_param6.grid(column=0, row=5, padx=10, pady=5)
entry_param6 = tk.Entry(tl)
entry_param6.grid(column=1, row=5, padx=10, pady=5)

# Button to run the algorithm
run_button = tk.Button(tl, text="Run Algorithm", command=on_run_button)
run_button.grid(column=0, row=6, columnspan=2, padx=10, pady=10)

# Label to display the results
result_label = tk.Label(tl, text="")
result_label.grid(column=0, row=7, columnspan=2, padx=10, pady=10)

# Start the Tkinter loop
tl.mainloop()

In [ ]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd

def update_stp():
    global stp
    try:
        stp_values = [float(entry.get()) for entry in entries]
        stp = [stp_values]
        contr = pd.DataFrame(stp, columns=columns_)
        messagebox.showinfo("Success", "Values updated successfully!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers.")

# Define the columns based on your requirements
columns_ = ['Col1', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6']

# Initialize stp with dummy data (will be updated by the user)
stp = [[0, 0, 0, 0, 0, 0]]

# Set up the Tkinter window
root = tk.Tk()
root.title("Update stp Values")

entries = []
for col in columns_:
    label = tk.Label(root, text=col)
    label.pack()
    entry = tk.Entry(root)
    entry.pack()
    entries.append(entry)

update_button = tk.Button(root, text="Update stp", command=update_stp)
update_button.pack()

root.mainloop()

In [ ]:
##finalversion
import tkinter as tk
from tkinter import messagebox, scrolledtext
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import statsmodels.discrete.discrete_model as sm
import time
import gc

# Initialize global variables
stp = [[0, 0, 0, 0, 0, 0]]
columns_ = ['Col1', 'Col2', 'Col3', 'Col4', 'Col5', 'Col6']
contr = pd.DataFrame(stp, columns=columns_)
ent_ = pd.DataFrame([[]], columns=columns_)  # Dummy DataFrame for 'ent_'
dt = pd.DataFrame()  # Placeholder for your actual DataFrame

def update_stp():
    global stp, contr
    try:
        stp_values = [float(entry.get()) for entry in entries]
        stp = [stp_values]
        contr = pd.DataFrame(stp, columns=columns_)
        messagebox.showinfo("Success", "Values updated successfully!")
    except ValueError:
        messagebox.showerror("Error", "Please enter valid numbers.")

def run_loop():
    global contr, dt, ent_

    condition_met = False
    start_time = time.time()
    time_limit = 300
    iteration = 0

    while not condition_met:
        try:
            # Track elapsed time
            elapsed_time = time.time() - start_time
            if elapsed_time > time_limit:
                result_text.insert(tk.END, "Time limit exceeded. Stopping the loop.\n")
                break

            iteration += 1
            result_text.insert(tk.END, f"Iteration: {iteration}, Time elapsed: {elapsed_time:.2f} seconds\n")

            # Reset index for contr DataFrame
            contr.reset_index(drop=True, inplace=True)

            # Concatenate and process DataFrames
            tt2 = pd.concat([ent_, contr], axis=1)
            tt2.fillna(0, inplace=True)
            tt2 = pd.DataFrame(tt2.sum()).T

            tt2_test = tt2[[col for col in columns_ if col in tt2.columns]]

            x_ = dt[[]]
            y_ = dt['opt']

            x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(x_, y_, test_size=0.9)
            model = LogisticRegression(penalty='l2', solver='liblinear', class_weight='balanced')
            model.fit(x_, y_)
            y_pred2 = model.predict(tt2)
            y_pred_prob2 = model.predict_proba(tt2)[:, 1]  # Get the probability of the positive class
            y_pred_score2 = model.score(x_test_3, y_test_3)

            # Inferência causal
            model = sm.Logit(y_train_3, x_train_3)
            co = model.fit(method='bfgs', cov_type='HC3').params
            coe = pd.DataFrame(co)
            coe.columns = ['coeff']
            exp_coeff = np.exp(coe) / (np.exp(coe) + 1)
            exp_coeff = pd.DataFrame(exp_coeff)
            exp_coeff.columns = ['exp_coeff']
            results = model.fit(method='bfgs', cov_type='HC3')
            results_summary = results.summary()
            results_as_html = results_summary.tables[1].as_html()
            df_results = pd.read_html(results_as_html, header=0, index_col=0)[0]
            df_results['exp_coeff'] = exp_coeff
            impact_var2 = df_results.loc[df_results['P>|z|'] < 0.05]

            results_list = []
            for i in impact_var2.index:
                value = impact_var2.loc[i, 'exp_coeff']

                if i in contr.columns:
                    correction = tt2_test[i].median()

                    if value < 0.5:
                        result = (value - 0.5) * correction
                    elif value > 0.5:
                        result = (value - 0.5) * correction
                    else:
                        result = 0

                    results_list.append({'Column': i, 'Alteração': result})

            results_df_ = pd.DataFrame(results_list)
            dd = results_df_.T
            new_header = dd.iloc[0]
            dd = dd[1:]
            dd.columns = new_header
            dd.reset_index(drop=True, inplace=True)

            common_columns = list(set(contr.columns) & set(dd.columns))
            contr_common = contr[common_columns]
            dd_common = dd[common_columns]

            combined_df = pd.concat([contr_common, dd_common], ignore_index=True)
            final = combined_df.sum().reset_index()
            tt = final.T
            new_header = tt.iloc[0]
            tt = tt[1:]
            tt.columns = new_header
            tt.reset_index(drop=True, inplace=True)

            contr.reset_index(drop=True, inplace=True)
            tt.reset_index(drop=True, inplace=True)

            common_columns = contr.columns.intersection(tt.columns)
            contr.update(tt[common_columns])
            contr.reset_index(inplace=True)
            contr.drop(columns=['index'], inplace=True)

            gc.collect()

            # Check if the condition is met
            if np.all(y_pred_prob2 >= 0.99) and np.all(y_pred2 == 1):
                condition_met = True

        except Exception as e:
            result_text.insert(tk.END, f"An error occurred: {e}\n")
            continue

    # After the loop ends, display the results
    result_text.insert(tk.END, "Final Results:\n")
    result_text.insert(tk.END, "Model Performance:\n")
    result_text.insert(tk.END, f"Accuracy Score: {y_pred_score2:.4f}\n")
    result_text.insert(tk.END, "\nPredicted Classes:\n")
    result_text.insert(tk.END, pd.DataFrame({'Predicted Class': y_pred2}).to_string() + "\n")
    result_text.insert(tk.END, "\nPredicted Probabilities:\n")
    result_text.insert(tk.END, pd.DataFrame({'Predicted Probability': y_pred_prob2}).to_string() + "\n")
    result_text.insert(tk.END, "\nResults DataFrame:\n")
    result_text.insert(tk.END, results_df_.to_string() + "\n")
    result_text.insert(tk.END, "\nUpdated 'contr' DataFrame:\n")
    result_text.insert(tk.END, contr.to_string() + "\n")

# Set up the Tkinter window
root = tk.Tk()
root.title("Processing Loop")

# Set up the input section for `stp`
tk.Label(root, text="Enter values for stp:").pack()
entries = []
for col in columns_:
    tk.Label(root, text=col).pack()
    entry = tk.Entry(root)
    entry.pack()
    entries.append(entry)

update_button = tk.Button(root, text="Update stp", command=update_stp)
update_button.pack()

run_button = tk.Button(root, text="Run Processing Loop", command=run_loop)
run_button.pack()

result_text = scrolledtext.ScrolledText(root, width=100, height=30)
result_text.pack()

root.mainloop()
